#Setup

In [ ]:
# !pip install xgboost catboost tsfresh seaborn
# !pip install pycaret[full]
# !pip  matplotlib==3.1.3
# !pip install pycaret-ts-alpha
# !pip install scipy>=1.5
# !pip install pystan==2.19.1.1
# !pip install Cython
# !pip install prophet
# !pip install -U kaleido
# !pip install numpy --upgrade
!cp "/content/drive/MyDrive/Colab Notebooks/Hackaton cajamar 2022/Cajamar_Water_Footprint.zip" "/content/data.zip"
!unzip -P  UH2022@CwF data

In [ ]:
%load_ext autoreload

In [ ]:
#@title imports
import pycaret
import pandas as pd
import logging
import matplotlib.pyplot as plt
from tqdm import tqdm
from prophet import Prophet

%matplotlib notebook
%matplotlib inline
import matplotlib
#https://towardsdatascience.com/time-series-feature-extraction-on-really-large-data-samples-b732f805ba0e
from tsfresh import extract_features, extract_relevant_features, select_features
from tsfresh.utilities.dataframe_functions import impute
from tsfresh.feature_extraction import ComprehensiveFCParameters
import numpy as np
from sklearn.ensemble import IsolationForest

In [ ]:
target='total'

In [ ]:
def negative_value_to_0(df):
        df['DELTAINTEGER']=np.where(df['DELTAINTEGER']<0,0,df['DELTAINTEGER'])
        return df
def negative_value_to_mean(df):
        df['DELTAINTEGER']=np.where(df['DELTAINTEGER']<0,df['DELTAINTEGER'].mean(),df['DELTAINTEGER'])
        return df    
def check_if_we_have_negative(df,column='DELTAINTEGER'):
    return df[df[column] < 0][column].any()
def clean_data(df):
    def clean_instances_with_minor_error(df,special_unique_negative_id):
            
        negative_instances=df[df['DELTAINTEGER']<0].sort_values(['ID','SAMPLETIME'])
        normal_unique_negative=list(negative_instances[~negative_instances.isin(special_unique_negative_id)]['ID'].unique())
        # print(df[df['ID'].isin(normal_unique_negative)].describe())
        df_normal_unique_negative=pd.DataFrame()
        for id_unique in tqdm(normal_unique_negative):
            df_per_id=df[df['ID']==id_unique]
            df_per_id=negative_value_to_0(df_per_id)
            df_per_id["ID"]
            df_normal_unique_negative=pd.concat([df_normal_unique_negative,df_per_id])#.reset_index()
        # print(df_normal_unique_negative.describe())
        return df_normal_unique_negative

    def clean_outliers_with_diverse_strategy(df,id_outliers_with_a_fews_peaks:dict):
        
        df_outlier_unique_negative=pd.DataFrame()
        for id_unique,args in tqdm(id_outliers_with_a_fews_peaks.items()):
            df_outlier_strategy_per_id=df[df['ID']==id_unique].sort_values(['ID','SAMPLETIME'])   
            # print(df_outlier_strategy_per_id.describe())     
            df_outlier_strategy_per_id['DELTAINTEGER']=np.where(   
                    df_outlier_strategy_per_id['DELTAINTEGER']<0,
                    df_outlier_strategy_per_id[(df_outlier_strategy_per_id['DELTAINTEGER']>0)&(df_outlier_strategy_per_id['DELTAINTEGER']<args['limit_max'])]['DELTAINTEGER'].mean(),
                    np.where(
                        df_outlier_strategy_per_id['DELTAINTEGER']>args['limit_max'],
                            args['limit_max']+0.2,
                            df_outlier_strategy_per_id['DELTAINTEGER']
                    )
                    ) 
            # print(df_outlier_strategy_per_id.describe()) 
            exist_negative=check_if_we_have_negative(df_outlier_strategy_per_id)
            if exist_negative:
                df_outlier_strategy_per_id=negative_value_to_mean(df_outlier_strategy_per_id)
            df_outlier_unique_negative=pd.concat([df_outlier_unique_negative,df_outlier_strategy_per_id])
        # print(df_outlier_unique_negative.describe())
        return df_outlier_unique_negative

    def strategy_inverse_to_clean_data(df,id_to_inverse_negative_values:list):

        df_inverse_strategy_unique_negative=pd.DataFrame()
        for id_unique in tqdm(id_to_inverse_negative_values):

            df_inverse_strategy_per_id=df[df['ID']==id_unique].sort_values(['ID','SAMPLETIME'])
            df_inverse_strategy_per_id['DELTAINTEGER']=abs(df_inverse_strategy_per_id['DELTAINTEGER'])

            exist_negative=check_if_we_have_negative(df_inverse_strategy_per_id)
            if exist_negative:
                print('ID ',id_unique,'already with negatives')
                df_inverse_strategy_per_id=negative_value_to_mean(df_inverse_strategy_per_id)
            df_inverse_strategy_unique_negative=pd.concat([df_inverse_strategy_unique_negative,df_inverse_strategy_per_id])
        return df_inverse_strategy_unique_negative

    special_unique_negative_id=[57, 301,374,379,493,635,845,
                            873, 907,1218,1280,1468,1487,
                            1506,1739,1873,1884, 2063,2121,
                            2711
                            ] #obtenidos del estudio exploratorio
    id_to_inverse_negative_values=[
                                    2121,
                                    2711,
                                    1873,
                                    379,
                                    1739,
                                    1468,]
    id_outliers_with_a_fews_peaks={
        57:{'limit_max':100},
        374:{'limit_max':2700},
        493:{'limit_max':200},
        635:{'limit_max':200},
        873:{'limit_max':200},
        907:{'limit_max':50},
        1218:{'limit_max':400},
        1280:{'limit_max':250},
        2063:{'limit_max':300},
        1884:{'limit_max':150},
        1506:{'limit_max':200},
        1487: { 'limit_max':150},
        301:{ 'limit_max':150},
        # cuando se pase esto a dias poner la media en ese fragmento que desaparece de este ID el id 845:
        #valores entre el 1 de mayo y el 8 de junio de 2019 se tienen que corregir de forma diaria
        845:{ 'limit_max':250},    
        }
    ids_with_negatives=negative_instances=df[df['DELTAINTEGER']<0].sort_values(['ID','SAMPLETIME'])['ID'].unique()
    df_with_negatives=df[df["ID"].isin(ids_with_negatives)].copy()
    df_without_negatives=df[~df["ID"].isin(ids_with_negatives)].copy() #pendiente ver los picos máximos para solucionarlo
    df_normal_unique_negative=clean_instances_with_minor_error(df_with_negatives,special_unique_negative_id)
    df_outlier_unique_negative=clean_outliers_with_diverse_strategy(df_with_negatives,id_outliers_with_a_fews_peaks)
    df_inverse_strategy_unique_negative=strategy_inverse_to_clean_data(df_with_negatives,id_to_inverse_negative_values)
    df_total=pd.concat([df_without_negatives,df_normal_unique_negative,df_outlier_unique_negative,df_inverse_strategy_unique_negative])
    
    return df_total.sort_values(['ID','SAMPLETIME'])

def outliers_after_groupby_period(df,outliers_fraction=0.01):
    method_forest=IsolationForest(contamination=outliers_fraction)
    df_after_outliers=pd.DataFrame()
    for id_unique in tqdm(df.ID.unique()):
        df_per_id=df[df['ID']==id_unique].copy()
        yhat = method_forest.fit_predict(df_per_id['total'].to_numpy().reshape(-1,1))
        mask = yhat != -1
        df_per_id['total']=np.where((mask) | (df_per_id.index> '2020-01-22'),
                                    df_per_id['total'],
                                    df_per_id['total'].shift(periods=7).fillna(method='bfill')
                            )
        df_after_outliers=pd.concat([df_after_outliers,df_per_id])
    return df_after_outliers

In [ ]:
def get_range_to_predic(start='2020-01-31',last_date='2020-01-31',periods=1,):
       
        freq='D'
        dates = pd.date_range(
            start=start,
            periods=periods,  # An extra in case we include start
            freq=freq)
    
        return dates
def add_new_date_per_id(df_per_id_but_without_column_id,date,id_unique):
    df_to_add_new_date=pd.DataFrame()
   
    df_to_add_new_date.index=get_range_to_predic(start=date,last_date=date)
    df_to_add_new_date.index = df_to_add_new_date.index.set_names(['date'])
    df_to_add_new_date.reset_index(inplace=True)
    
    df_to_add_new_date=df_to_add_new_date[df_to_add_new_date['date']==date]
      
    return df_to_add_new_date

def get_raw_data():
        data_path='/content/Modelar_UH2022.txt'
        df=pd.read_csv( data_path,sep="|")
        df=df[df['ID']<=25]
        # df=df[df['ID'].isin(list_of_id_with_tiny_data)]
        logging.info(df.shape)
        df.drop(columns=['READINGINTEGER','READINGTHOUSANDTH'],inplace=True)
        df.fillna(0,inplace=True) #check in future
        logging.info(df.shape)
        df.loc[:,'SAMPLETIME']=pd.to_datetime(df['SAMPLETIME'],errors='raise',format='%Y-%m-%d')
        return df

def get_data():
    
    df_raw=get_raw_data()
    df=clean_data(df_raw)
    # df['target']
    print(df.columns)
    df.rename(columns={'SAMPLETIME':'date'},inplace=True)
    df['total']=df['DELTAINTEGER']+df['DELTATHOUSANDTH']
    df.drop(['DELTAINTEGER','DELTATHOUSANDTH'],axis=1,inplace=True)
    print(df.columns)

    df_total=pd.DataFrame()
    for id_unique in tqdm(df.ID.unique(),desc='creating all the dates per ID',mininterval=50):
        df_per_id=df[df['ID']==id_unique]
        df_per_id_but_without_column_id=df_per_id.drop('ID',axis=1)
        list_of_df_with_new_dates=[]
        date='2020-01-31'
        if not df_per_id.date.isin([date]).any():
            # print('adding',date)
            df_with_last_date=add_new_date_per_id(df_per_id_but_without_column_id,date,id_unique)
            list_of_df_with_new_dates.append(df_with_last_date)
        # else:print(id_unique,'ya tiene la fecha',date)
        date='2019-02-01'
        if not df_per_id.date.isin([date]).any():
            # print('adding',date)
            df_with_start_date=add_new_date_per_id(df_per_id_but_without_column_id,date,id_unique)
            list_of_df_with_new_dates.append(df_with_start_date)
        # else:print(id_unique,'ya tiene la fecha',date)
        if list_of_df_with_new_dates:
            # print('list_of_df_with_new_dates')
            df_aux=pd.concat([df_per_id_but_without_column_id,*list_of_df_with_new_dates])
        else:    
            df_aux=df_per_id_but_without_column_id
            # print('no_new_date')
        # df_total.drop('ID',inplace=True,axis=1)
        df_aux=df_aux.resample('D',on='date').sum()
        df_aux['ID']=id_unique
        df_total=pd.concat([df_total,df_aux])#.reset_index()

    df_total_without_outliers=outliers_after_groupby_period(df_total)
    # pivot_by_dates=df_total_to_pivot.pivot(
    # index='date',
    # columns='ID',
    # values='total'
    # )
    return df_total_without_outliers

def plot_series(time, series,i, format="-", start=0, end=None):

    
    
    plt.plot(time[start:end], series[start:end], format,label=i)
    plt.xlabel("Date")
    plt.ylabel("Sales (Water)")
    plt.legend()

In [ ]:
df=get_data()

In [ ]:
df[df['ID']==3].plot()

In [ ]:
df.head()

In [ ]:
df_total_to_pivot=df.reset_index()
pivot_by_dates=df_total_to_pivot.pivot(
    index='date',
    columns='ID',
    values='total'
    )

In [ ]:
pivot_by_dates

mirar los casos positivos con esto
https://towardsdatascience.com/ways-to-detect-and-remove-the-outliers-404d16608dba

In [ ]:
#@title Researching
is_researching = True #@param {type:"boolean"}

id_researching=range(0,20)

#no modificar los días después del 22 de enero



# df_to_study=df[(df['DELTAINTEGER']>0)]['DELTAINTEGER'].quantile(0.75)+df_to_study_per_id[df_to_study_per_id['DELTAINTEGER']>0]['DELTAINTEGER'].std()*5
# for id_unique in tqdm(df.ID.unique()):
#     df_to_study=df_clean[df_clean['ID']==id_unique]
#     df_to_study=df_to_study[df_to_study['DELTAINTEGER']>(df_to_study['DELTAINTEGER'].quantile(0.75)+df_to_study['DELTAINTEGER'].std()*5)]

#     if not df_to_study.empty:
    
#         df_to_study=df[df['ID']==id_unique].sort_values(['ID','SAMPLETIME'])
#         print('ID',id_unique,'max',df_to_study['DELTAINTEGER'].max(),'min',df_to_study['DELTAINTEGER'].min())
#         p
#         min=df_to_study['DELTAINTEGER'].max()
#         print(df_to_study.describe())
#         # plt.ylim(min-5,abs(min)+5)
#         plt.show()
if is_researching:
    outliers_fraction=0.01
    method_forest=IsolationForest(contamination=outliers_fraction)
    df_after_outliers=pd.DataFrame()
    plt.figure(figsize=(20,5))
    for i,id_unique in enumerate(id_researching):
        df_per_id=df[df['ID']==id_unique].copy()
        
        df_per_id.head(20)
        yhat = method_forest.fit_predict(df_per_id['total'].to_numpy().reshape(-1,1))
        mask = yhat != -1
        plot_series(df_per_id.index,df_per_id['total'],f'todo {id_unique}')
        plot_series(df_per_id[mask].index,df_per_id[mask]['total'],f'true {id_unique}')
        plot_series(df_per_id[~mask].index,df_per_id[~mask]['total'],f'mask {id_unique}')
        plt.title('original')
        plt.show()
        plt.figure(figsize=(20,5))
        print('\n')
        df_per_id['total']=np.where((mask) | (df_per_id.index> '2020-01-22'),df_per_id['total'],
                df_per_id['total'].shift(periods=7).fillna(method='bfill')
                        
                        )
        plot_series(df_per_id.index,df_per_id['total'],f'todo {id_unique}')
        plot_series(df_per_id[mask].index,df_per_id[mask]['total'],f'true {id_unique}')
        plot_series(df_per_id[~mask].index,df_per_id[~mask]['total'],f'mask {id_unique}')
        plt.title('modified')
        plt.show()
        plt.figure(figsize=(20,5))
        df_after_outliers=pd.concat([df_after_outliers,df_per_id])
        print('\n'*3)
        if i>20:
            break
print(df_per_id[~mask])
    
# df_aux.reset_index(inplace=True)


##casos negativos

al agruparlo por día desaparecera el valor erroneo

In [ ]:
#@title comprobando que los negativos están arreglados

need_check_the_negatives_values = False #@param {type:"boolean"}
if need_check_the_negatives_values:
    df=get_raw_data()
    df_clean=clean_data(df)
    id_per_plot = 1 #@param {type:"number"}
    plt.figure(figsize=(20,5))
    special_unique_negative_id=[57, 301,374,379,493,635,845,
                                873, 907,1218,1280,1468,1487,
                                1506,1739,1873,1884, 2063,2121,
                                2711
                                ]
    for i, id_unique in tqdm(enumerate(special_unique_negative_id),desc="creating plots"):
        
        df_aux=df_clean[df_clean['ID'] == id_unique].sort_values(['ID','SAMPLETIME'])
        # bin=df_aux['group_by_mean_in_total'].unique()[0]
        plot_series(df_aux['SAMPLETIME'],df_aux['DELTAINTEGER'],id_unique)
        # min=df_aux['DELTAINTEGER'].min()
        if i %id_per_plot==0:
            # plt.ylim(min-5,abs(min)+5)
            plt.title(f'This id is {str(id_unique)}')
            plt.axhline(y = 0, color = 'k', linestyle = '-')
            plt.show()
            plt.figure(figsize=(20,5))

In [ ]:
import seaborn as sns

#drawing one figure with title and 3 axes below each other. Size and resolution are specified
f, axes = plt.subplots(4,1,figsize=(18,12),dpi=600,sharex=True)
plt.suptitle('Comparison of unscaled features at different scales',fontsize=22);

#drawing boxplots of three different scales, each to separate axis
max_id=100
sns.boxplot(data=pivot_by_dates.iloc[:,1:max_id], ax=axes[0])
sns.boxplot(data=pivot_by_dates.iloc[:,1:max_id], ax=axes[1]).set(ylim=(0,5000))
sns.boxplot(data=pivot_by_dates.iloc[:,1:max_id], ax=axes[2]).set(ylim=(0,2500))
sns.boxplot(data=pivot_by_dates.iloc[:,1:max_id], ax=axes[3]).set(ylim=(0,1000))

#rotating ticks of the shared x axis by 90 degrees. The shared x axis is located on axes[2]
for tick in axes[3].get_xticklabels():
        tick.set_rotation(90);

#setting y axis labels for each axis
for a in axes:
    a.set_ylabel('Unscaled values');

Con estos primeros 100 ID se puede observar que el rango de cada ID es muy variado, , desde 10000 de media algunos hasta muchos entre valores comprendidos menores a 1000. Se recomienda analizar el resto de ID de una forma más descriptiva, sin gráfica ya que esto solo representa el 5% aproximadamente de los datos

In [ ]:
describe_by_dates=pivot_by_dates.describe()
describe_by_dates

In [ ]:
describe_by_dates.T

In [ ]:
import seaborn as sns

#drawing one figure with title and 3 axes below each other. Size and resolution are specified
f, axes = plt.subplots(4,1,figsize=(18,12),dpi=600,sharex=True)
plt.suptitle('Comparison of unscaled features at different scales',fontsize=22);

#drawing boxplots of three different scales, each to separate axis
max_id=100
sns.boxplot(data=describe_by_dates.T, ax=axes[0]).set(ylim=(-5000,20000))
sns.boxplot(data=describe_by_dates.T, ax=axes[1]).set(ylim=(0,5000))
sns.boxplot(data=describe_by_dates.T, ax=axes[2]).set(ylim=(0,2500))
sns.boxplot(data=describe_by_dates.T, ax=axes[3]).set(ylim=(0,1000))

#rotating ticks of the shared x axis by 90 degrees. The shared x axis is located on axes[2]
for tick in axes[3].get_xticklabels():
        tick.set_rotation(90);

#setting y axis labels for each axis
for a in axes:
    a.set_ylabel('Unscaled values');

De esta forma se observa que la mayoría de los id tienen valores similares, con un rango intercuartilico de una amplitud de 500 en la media y desviación típica. Igual que el valor máximo tiene una gran amplitud de valores posibles y que el mínimo de normal es 0

In [ ]:
plot_series(describe_by_id.T.index,describe_by_id.T['mean'],'mean')

##Creating groups of ID per mean and per std

Empezamos creando dos grupos de ID iniciales, el primero de ellos mayor a 2000 y el segundo entre 1000 a 2000

In [ ]:
describe_by_dates

In [ ]:
group_2000=describe_by_dates.T[describe_by_dates.T['mean']>2000]
group_1000_2000=describe_by_dates.T[(describe_by_dates.T['mean']>1000)&(describe_by_dates.T['mean']<2000)]
group_0=describe_by_dates.T[describe_by_dates.T['mean']==0]
rest_df=describe_by_dates.T[(describe_by_dates.T['mean']<=1000)&(describe_by_dates.T['mean']>0)]
print(group_2000.shape)
print(group_1000_2000.shape)
print(group_0.shape)
rest_df['group_by_mean_in_total']=pd.qcut(rest_df['mean'],q=50,labels=range(2,52))
rest_df.groupby('group_by_mean_in_total').count()


In [ ]:
import seaborn as sns
sns.distplot(
    rest_df['std'], norm_hist=False, kde=False, bins=50, hist_kws={"alpha": 1}
).set(xlabel='Sale Price', ylabel='Count');

In [ ]:
group_2000=describe_by_dates.T[describe_by_dates.T['std']>2000]
group_1000_2000=describe_by_dates.T[(describe_by_dates.T['std']>1000)&(describe_by_dates.T['std']<2000)]
group_0=describe_by_dates.T[describe_by_dates.T['std']==0]
rest_df=describe_by_dates.T[(describe_by_dates.T['std']<=1000)&(describe_by_dates.T['std']>0)]
print(group_2000.shape)
print(group_1000_2000.shape)
print(group_0.shape)
rest_df['group_by_std_in_total']=pd.qcut(rest_df['std'],q=50,labels=range(2,52))
rest_df.groupby('group_by_std_in_total').count()

In [ ]:
import seaborn as sns
sns.distplot(
    rest_df['std'], norm_hist=False, kde=False, bins=50, hist_kws={"alpha": 1}
).set(xlabel='Sale Price', ylabel='Count');

Ya que los ID tienen tanta variabilidad entre ellos se ha decidido agrupar por grupos

In [ ]:
def create_cluster_by_mean(df,kind):
    #kind std or mean
  
    df_total_to_pivot=df.reset_index()
    pivot_by_dates=df_total_to_pivot.pivot(
    index='date',
    columns='ID',
    values='total'
    )
    describe_by_dates=pivot_by_dates.describe()
    group_2000=describe_by_dates.T[describe_by_dates.T[kind]>2000]
    group_1000_2000=describe_by_dates.T[(describe_by_dates.T[kind]>1000)&(describe_by_dates.T[kind]<2000)]
    group_0=describe_by_dates.T[describe_by_dates.T[kind]==0]
    rest_df=describe_by_dates.T[(describe_by_dates.T[kind]<=1000)&(describe_by_dates.T[kind]>0)]
    group_0[f'group_by_{kind}_in_total']=1
    rest_df[f'group_by_{kind}_in_total']=pd.qcut(rest_df[kind],q=50,labels=range(2,52))
    group_1000_2000['fgroup_by_{kind}_in_total']=52
    group_2000[f'group_by_{kind}_in_total']=53
    df_with_group_by_mean_in_total=pd.concat([group_0,rest_df,group_1000_2000,group_2000])[f'group_by_{kind}_in_total']
    return df_with_group_by_mean_in_total.reset_index()

df_with_group_by_mean_in_total=create_cluster_by_mean(df,'mean')

###Extracting features from tsfresh and group using pca

In [ ]:
import os
def extract_features_per_id_using_tsfresh(df,target='total'):
    #'This step we need use in colab because we have a problems with gpu'
    logging.info('creating features per id using tsfresh')
    path_features_tsfresh=r'D:\programacion\Repositorios\datathon-cajamar-2022\data\02_intermediate\features_tsfresh_per_id.csv'
    if os.path.isfile(path_features_tsfresh):
        X=pd.read_csv(path_features_tsfresh)
    else:
        from tsfresh import extract_features, extract_relevant_features, select_features
        from tsfresh.utilities.dataframe_functions import impute
        from tsfresh.feature_extraction import ComprehensiveFCParameters
        df=df[['ID',target,'date']]
        # We are very explicit here and specify the `default_fc_parameters`. If you remove this argument,
        # the ComprehensiveFCParameters (= all feature calculators) will also be used as default.
        # Have a look into the documentation (https://tsfresh.readthedocs.io/en/latest/text/feature_extraction_settings.html)
        # or one of the other notebooks to learn more about this.
        extraction_settings = ComprehensiveFCParameters()

        X = extract_features(df, column_id='ID', column_sort='date',
                            default_fc_parameters=extraction_settings,
                            # we impute = remove all NaN features automatically
                            impute_function=impute)
        X.reset_index(inplace=True)
        X.rename({'index':'ID'},inplace=True,axis=1)
        X.to_csv(path_features_tsfresh,index=False)
    return X

In [ ]:
df_tsfresh=extract_features_per_id_using_tsfresh(df.reset_index())

In [ ]:
from sklearn.decomposition import PCA, IncrementalPCA

def check_pca_in_df(df,extra_name:str='')->None:
    """remve id,date and total from the df before

    Args:
        df (_type_): without columns id, date or total
        extra_name (str, optional): _description_. Defaults to ''.
    """    
    def save_plot_explained_variance(name_path,pca):

        fig=plt.figure(figsize=(15,7))
        plt.plot(np.cumsum(pca.explained_variance_ratio_))
        plt.xlabel('number of components')
        plt.ylabel('cumulative explained variance')
        plt.show()
        plt.close()

    X=df.copy()#.drop(['ID','date','total'],axis=1)
    root_path=r'D:\programacion\Repositorios\datathon-cajamar-2022\data\08_reporting'
    # X=df_features_no_temporal.copy()
    # X.drop('total',inplace=True,axis=1)
    X.fillna(0,inplace=True)
    X.replace(np.inf,0,inplace=True)

    X.replace(-np.inf,0,inplace=True)
    n_components = 15
    ipca = IncrementalPCA(n_components=n_components, batch_size=256)
    X_ipca = ipca.fit_transform(X,)

    pca = PCA(n_components=n_components)
    X_pca = pca.fit_transform(X)
    
    path_ipca=os.path.join(root_path,extra_name+'_ipca.png')
    save_plot_explained_variance(path_ipca,ipca)
    path_ipca=os.path.join(root_path,extra_name+'_pca.png')
    save_plot_explained_variance(path_ipca,ipca)


In [ ]:
check_pca_in_df(df_tsfresh)

##Plotting a fews plots

In [ ]:
df_total_with_bins=pd.merge(df_with_group_by_mean_in_total,df,how='right',left_index=True,right_on='ID')


In [ ]:
#@title plot id per year
id_per_plot = 10 #@param {type:"number"}
plt.figure(figsize=(20,5))
for i, id_unique in tqdm(enumerate(df_total_with_bins.sort_values(['group_by_mean_in_total','date']).ID.unique()),desc="creating plots"):
    
    df_aux=df_total_with_bins[df_total_with_bins['ID'] == id_unique]
    bin=df_aux['group_by_mean_in_total'].unique()[0]
    plot_series(df_aux.index,df_aux['total'],id_unique)
    
    if i %id_per_plot==0:
        plt.title(f'This bin {str(bin)}')
        plt.show()
        plt.figure(figsize=(20,5))
    

En las gráficas se ve que el id 353 y 2711 hay que corregirlos, siendo en el 353 poniendo los picos a 0 y en el 2711 probablemente poniendolo en positivo el valor negativo y recalcular lo de los bins

##Análisis de dependencia por correlación entre los distintos ID

In [ ]:
pivot_by_dates.head()

In [ ]:

corr_matrix = pivot_by_dates.corr()

corr = ((corr_matrix + corr_matrix.T)/2 ).values                        # made symmetric
np.fill_diagonal(corr, 1)  

In [ ]:
corr

In [ ]:
import numpy as np
data_to_correlation=pivot_by_dates.copy()
# data_to_correlation.drop(['ID'],axis=1,inplace=True)
corr_matrix = data_to_correlation.corr()
 # Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
s = upper.unstack()
so = s.sort_values(kind="quicksort")
to_use_in_next_study = so[so>0.8]
to_use_in_next_study

Esto nos da información sobre variables que son similares a otras variables, por lo que deberíamos de utilizar esta información en el modelo en algún momento

##trends and stacionality

###using statsmodels

In [ ]:
import statsmodels.api as sm
from matplotlib import rcParams


In [ ]:
decomposition = sm.tsa.seasonal_decompose(pivot_by_dates[117].dropna())
# Store back the results
data = decomposition.seasonal
plot_series(data.index,data.values,117)
# if i%id_per_plots==0 and i>=id_per_plots:

plt.title(f'id between {str(i)}')
plt.show()
plt.figure(figsize=(20,15))

In [ ]:
id_per_plots=1
plt.figure(figsize=(20,15))
for i,ts in enumerate(pivot_by_dates.columns):
    decomposition = sm.tsa.seasonal_decompose(pivot_by_dates[ts].dropna())
    # Store back the results
    data = decomposition.seasonal
    plot_series(data.index,data.values,ts)
    if i%id_per_plots==0 and i>=id_per_plots:

        plt.title(f'id between {str(i)}')
        plt.show()
        plt.figure(figsize=(20,15))
    if i >=100:
        break

In [ ]:
id_per_plots=10
plt.figure(figsize=(20,15))
for i,ts in enumerate(pivot_by_dates.columns):
    decomposition = sm.tsa.seasonal_decompose(pivot_by_dates[ts].dropna())
    # Store back the results
    data = decomposition.trend
    plot_series(data.index,data.values,ts)
    if i%id_per_plots==0 and i>8:

        plt.title(f'id between {str(i)}')
        plt.show()
        plt.figure(figsize=(20,15))
    if i >=100:
        break

In [ ]:
id_per_plots=10
plt.figure(figsize=(20,15))
for i,ts in enumerate(pivot_by_dates.columns):
    decomposition = sm.tsa.seasonal_decompose(pivot_by_dates[ts].dropna())
    # Store back the results
    data = decomposition.resid
    plot_series(data.index,data.values,ts)
    if i%id_per_plots==0 and i>8:

        plt.title(f'id between {str(i)}')
        plt.show()
        plt.figure(figsize=(20,15))
    if i >=100:
        break

In [ ]:
from statsmodels.graphics import tsaplots
id_per_plots=10
plt.figure(figsize=(20,15))
for i,ts in enumerate(pivot_by_dates.columns):
    tsaplots.plot_acf(pivot_by_dates[ts], lags=90)
    
    plt.xlabel("Lag at k")
    plt.ylabel("Correlation coefficient")
    plt.title(f'id between {str(i)}')
    plt.show()
    plt.figure(figsize=(20,5))
    if i >=100:
        break

In [ ]:
from statsmodels.tsa.seasonal import STL
id_per_plots=10
for i,ts in enumerate(pivot_by_dates.columns):
    res=STL(pivot_by_dates[ts],).fit()
    fig=res.plot()
    fig.set_figheight(10)
    fig.set_figwidth(20)
    plt.xlabel("Lag at k")
    plt.ylabel("Correlation coefficient")
    plt.title(f'id between {str(i)}')
    plt.show()
    print('\n')
    if i >=100:
        break

In [ ]:
a

###using prophet

In [ ]:
pivot_by_dates.plot()

In [ ]:
df_prophet=pd.DataFrame()
df_aux=pivot_by_dates[117]
df_prophet[['ds','y']]=df_aux.reset_index()

In [ ]:
df_prophet

Solo mostramos los 20 primeros

In [ ]:
def split_data(data, split_date,column_date=None):
    if column_date:
        return data[data[column_date] <= split_date].copy(), \
           data[data[column_date] >  split_date].copy()
           
    else: return data[data.index <= split_date].copy(), \
           data[data.index >  split_date].copy()


In [ ]:
pivot_by_dates[3].plot()

In [ ]:
from sklearn.metrics import mean_squared_error
def rmse(y_true,y_pred):
    error=mean_squared_error(y_true,y_pred,squared=False)
    return error
error_df=pd.DataFrame()
for i,ts in tqdm(enumerate(pivot_by_dates.columns)): 
    
    df_prophet=pd.DataFrame()
    df_aux=pivot_by_dates[ts]
    train, test = split_data(df_aux, '2020-01-24') 
    df_prophet[['ds','y']]=train.reset_index()
    m = Prophet(yearly_seasonality=False,
        weekly_seasonality=True,
        daily_seasonality=False
        )
    m.fit(df_prophet)
    future=m.make_future_dataframe(7)
    forecast = m.predict(future)
    forecast['yhat']=np.where(forecast['yhat']<0,
                              0,
                              forecast['yhat'])
    train_forecast, test_forecast = split_data(forecast, '2020-01-24','ds') 
    error=rmse(test,test_forecast.yhat)
    error_df.loc[ts,'Prophet_without_exogenous']=error
    print(error_df.head())
    plot_series(train.index,train,f'{ts} train_true')
    plot_series(test.index,test,f'{ts} test_true')
    plot_series(train_forecast.ds,train_forecast['yhat'],f'{ts} train_predict')
    plot_series(test_forecast.ds,test_forecast['yhat'],f'{ts} test_predict')
    plt.show()
    # fig1 = m.plot(forecast)
    # fig1.suptitle('id '+str(ts))
    # fig1.show()
    # print("\n"*1)
    # fig2 = m.plot_components(forecast)
    # fig2.suptitle('id '+str(ts))
    # print("\n"*3)
    if i>20:
        break

In [ ]:
error_df.head(20)

In [ ]:
error_df.sum()

### with pycaret --> time series

In [ ]:
from pycaret import time_series

In [ ]:
fh=7

In [ ]:
for i,ts in tqdm(enumerate(pivot_by_dates.columns),mininterval=5,desc='creating_models'): 
    # df_to_time_regression_original=df[df['ID']==id_unique].copy()
    df_to_time_regression_original=pivot_by_dates[ts]
    train, test = split_data(df_to_time_regression_original, '2020-01-24') 
    
    time_series.setup(
    data=train, fh=fh,
    enforce_exogenous=False,
    # Set defaults for the plots ----
    # fig_kwargs={"renderer": "notebook", "width": 1000, "height": 600},
    session_id=42,
    
    )
    best=time_series.compare_models(turbo=False,sort='RMSE',round=2,
                                    exclude=[
                                             'huber_cds_dt',
                                             'tbats',
                                             'bats',
                                             'auto_arima'])

    print('This is the id:', id_unique)
    path=r'D:\programacion\Repositorios\datathon-cajamar-2022\data\06_models\best_model_'+str(id_unique)

    
    best=time_series.finalize_model(best)
    time_series.save_model(best,path)

    predictions=time_series.predict_model(best,fh=7,return_pred_int=False,round=2)
    
    error=rmse(test,test_forecast.yhat)
    error_df.loc[ts,'pycaret_time_series_without_exogenous']=error
    plot_series(train.index,train,f'{ts} train_true')
    plt.show()
    plot_series(test.index,test,f'{ts} test_true')
    # plot_series(train_forecast.ds,train_forecast['yhat'],f'{ts} train_predict')
    plot_series(predictions.index,predictions,f'{ts} test_predict')

    if id_unique>20:
        break

In [ ]:
error=rmse(test,test_forecast.yhat)
error_df.loc[ts,'pycaret_time_series']=error

In [ ]:
error_df

#Regression
###solo usar si creamos distintas caracteriticas con prophet por ejemplo

#as regression problem

##features

In [ ]:
target= 'total'
key_metric='RMSE'

In [ ]:
#@title extract features

from src.create_the_output_file import create_outputfile
from src.generate_df import generate_df
from src.pipeline import  train_systems
from src.predict import predict
from src.utils import extract_n_relevant_features
from src.create_features import create_features,extract_features_per_id_using_tsfresh

In [ ]:
#@title Variables


In [ ]:
df=generate_df('/content/Modelar_UH2022.txt','W')

In [ ]:
df.head()

In [ ]:
aggregated=create_features(df,'W')

In [ ]:
# Create correlation matrix
data_without_total=aggregated.copy()
data_without_total.drop(target,axis=1,inplace=True)
corr_matrix = data_without_total.corr()
 # Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
to_drop = [column for column in upper.columns if any(upper[column] > 0.8)]
print(len(to_drop))

In [ ]:
for value_max in range(60,100,5):
    to_drop = [column for column in upper.columns if any(upper[column] >value_max/100)]
    print('value_max',value_max,'columns to drop',len(to_drop))

In [ ]:
#@title to get df_tsfresh
is_needed_df_tsfresh = True #@param {type:"boolean"}
if is_needed_df_tsfresh:
    df_tsfresh=extract_features_per_id_using_tsfresh(df)
    df_tsfresh.reset_index(inplace=True)
    df_tsfresh.shape

In [ ]:
#@title Outliers
from sklearn.ensemble import IsolationForest
outliers_fraction=0.1
method_forest=IsolationForest(contamination=outliers_fraction)
df_to_use=aggregated[['total','ID','date']]
df_aux=pd.DataFrame()
for id_unique in df_to_use.ID.unique():
    df_per_id=df_to_use[df_to_use['ID']==id_unique].copy()
    add_later=df_per_id['date']
    df_per_id.drop('date',axis=1,inplace=True)
    data_to_correction=aggregated[aggregated['ID']==id_unique]['14d_roll_mean']
    yhat = method_forest.fit_predict(df_per_id)
    mask = yhat != -1
    df_per_id['total']=np.where(mask,
                                df_per_id.total,
                                data_to_correction
                            )
    df_per_id.loc[:,'date']=add_later
    df_aux=pd.concat([df_aux,df_per_id],axis=0)
    
df_aux.reset_index(inplace=True)

In [ ]:
aggregated.shape

In [ ]:
# Create correlation matrix
data_to_correlation=df_features_no_temporal.copy()
data_to_correlation.drop(['ID'],axis=1,inplace=True)
corr_matrix = data_to_correlation.corr()
 # Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
to_use_in_next_study = [column for column in upper.columns if any(upper[column] < 0.1)]

In [ ]:
# Drop Marked Features
to_drop=[]
# to_drop = [column for column in upper.columns if any(upper[column] > 0.7)]
print(len(to_drop))
print(aggregated.shape)
aggregated_after_drop = aggregated.drop(to_drop, axis=1)
print(aggregated_after_drop.shape)


In [ ]:
train, test = split_data(aggregated_after_drop, '2020-01-15') 
train_aux, test_aux = split_data(df_aux, '2020-01-15') 

In [ ]:
train=aggregated_after_drop

In [ ]:
train['total']=train_aux['total']

In [ ]:
for id in aggregated.ID.unique():
    train_per_id=train[train['ID']==id]
    test_per_id=test[test['ID']==id]
    plt.figure(figsize=(20,10))
    plt.xlabel('date')
    plt.ylabel('Water_used')
    plt.plot(train_per_id['date'],train_per_id[target],label='train')
    plt.plot(test_per_id['date'],test_per_id[target],label='test')
    plt.title(id)
    plt.legend()
    plt.show()
    if id>10:
        break

##train

In [ ]:
from pycaret.regression import *

In [ ]:
#@title Pycaret setup
reg = setup(data = train, 
             target = target,
             numeric_imputation = 'mean',
            ignore_features=['ID','date',
                            #  'dayofmonth',
                            #  'month'
                            #  'dayofweek'
                             ],
            #faltará añadir más datos en numericos de estacionalidd, tendencia, etc por cada ID mediante prophet
             categorical_features = [
                                     'dayofweek',
                                    #  'quarter','month',
                                    #  'year',
                                    #  'dayofyear',
                                     'dayofmonth',
                                    #  'weekofyear',
                                    #  'ID'
                                     ]  , 
            normalize=True,
            transformation = True, transform_target = False, 
            transform_target_method='yeo-johnson',
                  combine_rare_levels = True, rare_level_threshold = 0.1,
                #   remove_multicollinearity = True, multicollinearity_threshold = 0.95, 
            # feature_selection=True,
            create_clusters=True,
            cluster_iter=5,
            # pca=True,
             silent = True)

In [ ]:
#@title Models available
see_models_available = False #@param {type:"boolean"}
if see_models_available:
    all_models = models()
    print(all_models)

In [ ]:
# returns best models - takes a little time to run
top = compare_models(sort=key_metric,exclude=[
                                              'rf',
                                              'huber',
                                            #   'br',
                                              'ada',
                                              'et',
                                              'xgboost',
                                              'catboost',
                                            #   'knn',
                                            #   'gbr'
                                     
                                                # 'ridge',
                                                # 'lasso',
                                                # 'omp',
                                                # 'par',
                                                # 'en',
                                                # 'lar'
                                              ],
                     round=2,
                     turbo=True
                     )

In [ ]:
#we create a model using light gbm
lightgbm = create_model('lightgbm')

In [ ]:
tuned_lightgbm = tune_model(lightgbm,early_stopping=True,optimize='rmse') 

In [ ]:
evaluate_model(lightgbm)

In [ ]:
evaluate_model(tuned_lightgbm)

In [ ]:
plot_model(lightgbm)

In [ ]:
plot_model(lightgbm, plot = 'error')

In [ ]:
def extract_n_relevant_features(model,n=20):
    X_train=get_config('X_train',)
    feats = {} # a dict to hold feature_name: feature_importance
    for feature, importance in zip(X_train.columns, model.feature_importances_):
        feats[feature] = importance #add the name/value pair 

    importances = pd.DataFrame.from_dict(feats, orient='index').rename(columns={0: 'Gini-importance'})
    return importances.sort_values(by='Gini-importance',ascending=False).head(n)#.plot(kind='bar', rot=45)


In [ ]:
plot_model(lightgbm, plot='feature_all')


In [ ]:
n=30
features=extract_n_relevant_features(lightgbm,n)
print(features.head(n))
features.index.to_list()

In [ ]:
plot_model(tuned_lightgbm, plot='feature_all')


In [ ]:
n=35
features=extract_n_relevant_features(tuned_lightgbm,n)
print(features.head(n))
features.index.to_list()

In [ ]:
print(dir(tuned_lightgbm))

In [ ]:
plot_model(tuned_lightgbm, plot='rfe')

In [ ]:
predict_model(tuned_lightgbm)

In [ ]:
final_lightgbm = finalize_model(tuned_lightgbm)

In [ ]:
unseen_predictions = predict_model(final_lightgbm, data=test)
unseen_predictions.head()
unseen_predictions.loc[unseen_predictions['Label'] < 0, 'Label'] = 0 #removing any negative values

In [ ]:
def plot_series(time, series,i, format="-", start=0, end=None):

    
    
    plt.plot(time[start:end], series[start:end], format,label=i)
    plt.xlabel("Date")
    plt.ylabel("Sales (Water)")
    plt.legend()
def plot_results(test,unseen_predictions,stop:int=5,label:str='Baseline'):
    for id in test.ID.unique():
        plt.figure(figsize=(15,7))
        test_per_id=test[test['ID']==id]
        unseen_predictions_per_id=unseen_predictions[unseen_predictions['ID']==id]
        plot_series(test_per_id['date'], test_per_id[target],f"True {id}")
        #plot_series(train['ds'],train['y'])
        plot_series(test_per_id['date'], unseen_predictions_per_id['Label'],f"{label} {id}")
        plt.show()
        print('\n')
        if id>=stop:
            break

In [ ]:
plot_results(test,unseen_predictions,15)

###Blending Models

In [ ]:
gbr = create_model('gbr', verbose = False)
# rf = create_model('rf', verbose = False)
lightgbm = create_model('lightgbm', verbose = False)
xgb = create_model('xgboost',verbose=False)

In [ ]:
# tuned_rf = tune_model(rf)
tuned_gbr = tune_model(gbr)
tuned_lightgbm = tune_model(lightgbm)
tuned_xgb = tune_model(xgb)

In [ ]:
blend_specific = blend_models(estimator_list = [tuned_lightgbm,tuned_gbr,tuned_xgb])

In [ ]:
blend_specific

In [ ]:
predict_model(blend_specific);

In [ ]:
final_model = finalize_model(blend_specific)

In [ ]:
unseen_predictions_2 = predict_model(final_model, data=test, round=0)
unseen_predictions_2.loc[unseen_predictions_2['Label'] < 0, 'Label'] = 0
unseen_predictions_2.head()

In [ ]:
plot_results(test,unseen_predictions_2,label='blend')

###Stacking

In [ ]:
stack_1 = stack_models([tuned_lightgbm,tuned_gbr,tuned_xgb])
predict_model(stack_1);
final_stack_1 = finalize_model(stack_1)
unseen_predictions_3 = predict_model(final_stack_1, data=test, round=0)

In [ ]:
unseen_predictions_3.loc[unseen_predictions_3['Label'] < 0, 'Label'] = 0
unseen_predictions_3.head(4)

In [ ]:
plot_results(test,unseen_predictions_3,label='stacking')

In [ ]:
def plot_all_results(test,unseen_predictions_dict:dict,stop:int=5,):
    for id in test.ID.unique():
        plt.figure(figsize=(15,7))
        test_per_id=test[test['ID']==id]
        # unseen_predictions_per_id=unseen_predictions[unseen_predictions['ID']==id]
        plot_series(test_per_id['date'], test_per_id[target],f"True {id}")
        #plot_series(train['ds'],train['y'])
        for label,df_unseen in unseen_predictions_dict.items():
            df_unseen=df_unseen[df_unseen['ID']==id]
            plot_series(test_per_id['date'], df_unseen['Label'],f"{label} {id}")
        plt.show()
        print('\n')
        if id>=stop:
            break

In [ ]:
unseen_predictions_dict={
    'baseline':unseen_predictions,
    'blend': unseen_predictions_2, 
    'stacking': unseen_predictions_3, 
}
plot_all_results(test,unseen_predictions_dict)

Train only timeseries

##Predictions

necesito las cuatro formulas que uso para extraer características

basic_temporal_features -> hecho
extract_features_per_id_using_tsfresh -> no depende del tiempo por lo que coger el df
extract_features_from_prophet
extra_features_manually_no_temporal_variables -> no depende del tiempo por lo que coger el df
extra_features_manually_temporal_variables -> hecho


In [ ]:
test

In [ ]:
start='2020-02-01'
last_date='2020-02-14'
freq='D'
periods=14

In [ ]:
dates = pd.date_range(
            start=start,
            periods=periods,  # An extra in case we include start
            freq=freq)
unique_ids=df.ID.unique()
# dates = dates[dates > last_date]  # Drop start if equals last_date
# dates = dates[:periods]  # Return correct number of periods

In [ ]:
df_aux=pd.DataFrame()
for id in unique_ids:
    df_per_id=pd.DataFrame()
    df_per_id.index=dates
    df_per_id.index = df_per_id.index.set_names(['date'])
    df_per_id.reset_index(inplace=True)
    df_per_id['ID']=id
    df_per_id['year']=df_per_id['date'].dt.year
    df_per_id['month']=df_per_id['date'].dt.month
    df_per_id['day']=df_per_id['date'].dt.day



In [ ]:
df_to_test=pd.concat([df,df_per_id,])

In [ ]:
df_to_test=df_to_test[df_to_test['date']<'2020-02-02']

In [ ]:
df_to_test[df_to_test['date']=='2020-02-01']

In [ ]:
df_to_test=basic_temporal_features(df_to_test)
features_using_prophet=extract_features_from_prophet(df_to_test,is_to_predict=True) #habría que ir cambiando el total y volviendo a iterar esto por cada día
# features_manually_no_temporal=extra_features_manually_no_temporal_variables(df_to_test)
features_manually_temporal_variables=extra_features_manually_temporal_variables(df_to_test)
#añadir la suma del mes anterior
#añadir la suma de la semana anterior
#añadir la suma del trimestre anterior

#add data from tsfresh y from prophet (en ambos casos se hará por id y luego se mergeará a ese ID)
X = df_to_test[['dayofweek','quarter','month',
        # 'year',  #only we have a year, and that is not enought to the model learn what hapen to the next year
        'dayofyear','dayofmonth','weekofyear','total','ID','date']]

# X=pd.merge(X,features_tsfresh_per_id,right_index=True,left_on='ID')
# X=pd.merge(X,features_using_prophet,left_on=['ID','date'],right_on=['ID','ds'])
# X=pd.merge(X,features_manually_no_temporal,right_on='ID',left_on='ID')
df2=pd.merge(X,features_manually_temporal_variables,right_on=['ID','date'],left_on=['ID','date'])

In [ ]:
df2[df2['date']=='2020-02-01']

In [ ]:
df2.tail()

In [ ]:
features_using_prophet=extract_features_from_prophet(df_to_test,is_to_predict=True)

In [ ]:
features_using_prophet

In [ ]:
df_to_test=extra_features_manually_temporal_variables(df_to_test)

In [ ]:
b=df_to_test[df_to_test['ID']==4]
b

In [ ]:
df_per_id=df_to_test[df_to_test['ID']==4]
# df_per_id.fillna(0,inplace=True)

In [ ]:
df_per_id["1d_roll_sum"] = df_per_id[target].rolling(window=1,closed='left').sum().fillna(method='bfill')

In [ ]:
df_per_id['shifted_1']=df_per_id[target].shift(periods=1).fillna(method='bfill')

In [ ]:
df_per_id

In [ ]:
a[a['date']=='2020-02-01']

In [ ]:
predict_model(final_model, data=test, round=0)

#Time series

In [ ]:
import time
import numpy as np
import pandas as pd
from pycaret.internal.pycaret_experiment import TimeSeriesExperiment

from pycaret.time_series import TSForecastingExperiment
# exp=from pycaret.internal.pycaret_experiment import TimeSeriesExperiment

In [ ]:
from pycaret import time_series
import numpy as np
from pipeline import split_data

In [ ]:
df=get_data()

In [ ]:
df_correction=cheking_outliers(df.copy())

In [ ]:
fh = 7 # or alternately fh = np.arange(1,13)
fold = 5
target='total'

In [ ]:
from typing import Tuple

In [ ]:
results=pd.DataFrame()
for id_unique in tqdm(df.ID.unique(),mininterval=5,desc='creating_models'):
    df_to_time_regression_original=df[df['ID']==id_unique].copy()
    df_correction=cheking_outliers(df.copy())
    df_correction_per_id=df_correction[df_correction['ID']==id_unique].copy()
    df_to_time_regression=df_to_time_regression_original.copy()
    df_to_time_regression.index=df_to_time_regression.date
    df_correction_per_id.index=df_to_time_regression.date

    df_train_correction,df_test_correction=split_data(df_correction_per_id.copy(), '2020-01-15')
    
    df_train,df_test=split_data(df_to_time_regression.copy(), '2020-01-15') 
    print(df_train.head())
    print(df_train_correction.head())
    df_train['total']=df_train_correction['total']
    print(df_train.head())
    df_train.drop(['date','ID'],inplace=True,axis=1)
    df_test.drop(['date','ID'],inplace=True,axis=1)
    time_series.setup(
    data=df_train, target=target, fh=fh,
    enforce_exogenous=False,
    # Set defaults for the plots ----
    # fig_kwargs={"renderer": "notebook", "width": 1000, "height": 600},
    session_id=42,
    
    )
    best=time_series.compare_models(turbo=False,sort='RMSE',round=2,
                                    exclude=[
                                             'huber_cds_dt',
                                             'tbats',
                                             'bats',
                                             'auto_arima'])

    print('This is the id:', id_unique)
    path=r'D:\programacion\Repositorios\datathon-cajamar-2022\data\06_models\best_model_'+str(id_unique)

    time_series.save_model(best,path)
    best=time_series.finalize_model(best)
    predictions=time_series.predict_model(best,fh=14,return_pred_int=False,round=2)
    df_test_to_merge=df_test.head(14)
    df_test_to_merge.index=predictions.index
    results_temporal=pd.DataFrame()
    results_temporal=pd.merge(df_test_to_merge,predictions,left_index=True,right_index=True)
    results_temporal['ID']=id_unique

    results=pd.concat([results,results_temporal])
    if id_unique>3:
        break


In [ ]:
result_with_date=results.reset_index()
result_with_date.rename({'index':'date'},inplace=True)

In [ ]:
for id_unique in result_with_date.ID.unique():
    results_id=result_with_date[result_with_date['ID']==id_unique]
    plt.figure(figsize=(17,7))
    plot_series(results_id.index.to_list(), results_id['total'],f"True {id_unique}")
            #plot_series(train['ds'],train['y'])
    plot_series(results_id.index.to_list(), results_id['y_pred'],f"prediction {id_unique}")
    plt.show()

In [ ]:
from sklearn.metrics import mean_squared_error
errors={}
errors['ID']=[]
errors['RMSE']=[]
for id_unique in result_with_date.ID.unique():
    error_per_id=pd.DataFrame()
    results_id=result_with_date[result_with_date['ID']==id_unique]
    
    error=mean_squared_error(results_id['y_pred'],results_id['total'],squared=False)
    errors['ID'].append(id_unique)
    errors['RMSE'].append(error)
    
    # error_per_id['ID']=id_unique
    # error_per_id['RMSE']=error
    # errors_per_id=pd.concat([errors_per_id,error_per_id])

In [ ]:
pd.DataFrame(errors)

In [ ]:
errors_per_id

In [ ]:
time_series.plot_model(plot="diagnostics")

In [ ]:
# Plot the target variable along with any exogenous variables ----
# Becasue the data is quite huge, plotting an interactive plot can slow down the notebook.
# Hence, we will revert to a static renderer for this plot
time_series.plot_model(fig_kwargs={"renderer": "png", "height": 1200})

In [ ]:
# Plots original data with first difference (order d = 1) by default
time_series.plot_model(
    plot="diff",
    data_kwargs={"acf": True, "pacf": True, "periodogram": True}    
)

In [ ]:
time_series.plot_model(
    plot="diff",
    data_kwargs={"lags_list":[1, [1, 24]],"acf": True, "pacf": True, "periodogram": True}    
)

In [ ]:
time_series.plot_model(plot="ccf", fig_kwargs={"height": 1000})

##model univariate

In [ ]:
time_series.plot_model(best,'forecast',fig_kwargs={"renderer": "png", "height": 1200}  )

In [ ]:
eda.plot_model(plot="decomp_classical")
eda.plot_model(plot="decomp_classical", data_kwargs={'type': 'multiplicative'})
eda.plot_model(plot="decomp_stl")

In [ ]:
# Show the train-test splits on the dataset
# Internally split - len(fh) as test set, remaining used as test set
eda.plot_model(plot="train_test_split",fig_kwargs={"renderer": "png", "height": 1200} )

# Show the Cross Validation splits inside the train set
eda.plot_model(plot="cv",fig_kwargs={"renderer": "png", "height": 1200})

In [ ]:
# Plot diagnostics
eda.plot_model(plot="diagnostics",fig_kwargs={"renderer": "png", "height": 1200} )

In [ ]:
# Plot differences along with diagnostics such as ACF and PACF

# Row 1: Original
# Row 2: d = 1
# Row 3: d = 2
eda.plot_model(plot="diff", data_kwargs={"order_list": [1, 2], "pacf": True},fig_kwargs={"renderer": "png", "height": 1200} )

# Row 1: Original
# Row 2: d = 1
# Row 3: First (d = 1) then (D = 1, s = 12)
#   - Corresponds to applying a standard first difference to handle trend, and
#     followed by a seasonal difference (at lag 12) to attempt to account for
#     seasonal dependence.
# Ref: https://www.sktime.org/en/v0.8.0/api_reference/modules/auto_generated/sktime.transformations.series.difference.Differencer.html
eda.plot_model(plot="diff", data_kwargs={"lags_list": [[1], [1, 12]], "acf": True, "pacf": True},fig_kwargs={"renderer": "png", "height": 1200} )

In [ ]:
eda.plot_model(best, plot = 'insample',fig_kwargs={"renderer": "png", "height": 1200} )

In [ ]:
df=get_data()
print(df.head())


In [ ]:
len(df.ID.unique())

#Output format

In [ ]:
df_original=df.copy()
df_original.shape

In [ ]:
df=get_data()

In [ ]:
basic_temporal_features(df).total.isna().sum()

In [ ]:
test[['ID','2d_roll_sum','shifted_diff_1','shifted_diff_2']]

In [ ]:
print(df_full_to_get_temporal_features.shape)

In [ ]:
#we need predict 14 days of February 7 days to predict and 2 weeks
def get_range_to_predic():
    start='2020-02-01'
    last_date='2020-02-14'
    freq='D'
    periods=14
    dates = pd.date_range(
        start=start,
        periods=periods,  # An extra in case we include start
        freq=freq)

    return dates

# model=regression.load_model(model)
dates=get_range_to_predic()
unique_id=df.ID.unique()
for day_to_predict in dates:
    print(day_to_predict)
    df_aux=pd.DataFrame()
    for id_unique in unique_id:
        df_per_id=pd.DataFrame()
        df_per_id.index=dates
        df_per_id.index = df_per_id.index.set_names(['date'])
        df_per_id.reset_index(inplace=True)
        df_per_id['ID']=id_unique

        df_per_id=df_per_id[df_per_id['date']==day_to_predict]

        df_aux=pd.concat([df_aux,df_per_id])
    df_aux.loc[:,'date']=pd.to_datetime(df_aux['date'],errors='raise',format='%Y-%m-%d')

    df_to_predict=basic_temporal_features(df_aux)

    df_full_to_get_temporal_features=pd.concat([df_to_predict,df])
    df_full_to_get_temporal_features.sort_values(['ID','date'],inplace=True)

    df_full=create_features(df_full_to_get_temporal_features)
    df_to_predict=df_full[df_full['date']==pd.to_datetime(day_to_predict)]
    logging.debug(df_to_predict.shape)
    results=pd.DataFrame()
    

        
    df_to_predict_per_id=df_to_predict[df_to_predict['ID']==id_unique]
    results=predict_model(tuned_lasso,df_to_predict)
    

    print(results.shape)
    print(results[['ID','total','date','Label']].tail(15))
    print(results[results['date']==pd.to_datetime(day_to_predict)][['ID','total','date']].tail(5))
    results['total']=results['Label']
    results.drop('Label',axis=1,inplace=True)
    print(df.shape)
    df=pd.concat([df,results])
    print(df.shape)
    df.sort_values(['ID','date'],inplace=True)
predictions=df



In [ ]:
a=df[df['date']>='2020-01-30']

In [ ]:
a.head(7)

In [ ]:
b=aggregated[aggregated['date']>='2020-01-30']


In [ ]:
b.head(7)

In [ ]:
df_full_to_get_temporal_features.dropna(inplace=True)

In [ ]:
output_data.head(20)

In [ ]:
one_id=output_data[output_data['ID']==0].copy()
# one_id=one_id[(one_id['date']>='2020-01-16') & (one_id['date']<='2020-01-22')]
# one_id.head(20)

In [ ]:
one_id.Label.sum()

In [ ]:
algo=output_data.pivot(
    index='ID',
    columns='date',
    values='Label'
)
algo

In [ ]:
def create_the_sum_between_dates(df,init_date:str,end_date:str):
    return df[(df['date']>=init_date) & (df['date']<=end_date)].Label.sum()

def create_the_two_rows_extra_per_id(df):
    df_aux=pd.DataFrame()
    df=df[['ID','Label','date']]
    init_date_1='2020-01-01' #cambiar por febrero
    end_date_1='2020-01-07'
    init_date_2='2020-01-18'
    end_date_2='2020-01-25'
    for id in tqdm(df.ID.unique(),desc='features temporal variables per id'):
            
        df_per_id=df[df['ID']==id]
        df_per_id['Semana_1']=create_the_sum_between_dates(df_per_id,init_date_1,end_date_1)
        df_per_id['Semana_2']=create_the_sum_between_dates(df_per_id,init_date_2,end_date_2)
            
        df_per_id=df_per_id[['ID','Semana_1','Semana_2']].drop_duplicates()
        df_aux=pd.concat([df_per_id,df_aux])    
    return df_aux

def create_pivot_table(df):
    df=df.pivot(
        index='ID',
        columns='date',
        values='Label'
    )
    # replace the Month with year 
    # df = df.rename(columns={"date":"ID"})

    # drop first column
    # df = df.iloc[1:].reset_index(drop=True)
    # df.reset_index(inplace=True)
    # df.drop('date',inplace=True,axis=1)
    return df


def create_output_df(df):
    pivot_table=create_pivot_table(df)
    values_per_week=create_the_two_rows_extra_per_id(df)
    df=pd.merge(pivot_table,values_per_week,on='ID')
    return df

def create_outputfile(df,path_output='Cajamar_Universitat Politécnica de Valencia (UPV)_CanarIAs_1.txt'):
    #Sin cabecera ni nombres de filas.
    # Constará de 2.747 filas con 10 columnas cada fila:
    # • ID: ordenado de forma ascendente
    # • Dia_1: Predicción para el día 01/02/2020
    # • Dia_2: Predicción para el día 02/02/2020
    # • Dia_3: Predicción para el día 03/02/2020
    # • Dia_4: Predicción para el día 04/02/2020
    # • Dia_5: Predicción para el día 05/02/2020
    # • Dia_6: Predicción para el día 06/02/2020
    # • Dia_7: Predicción para el día 07/02/2020
    # • Semana_1: Predicción para la semana del 01/02 al 07/02/2020, ambos inclusive
    # • Semana_2: Predicción para la semana del 08/02 al 14/02/2020, ambos inclusive
    #Separando campos con “|”, el valor de la predicción en litros, y los decimales con “.” (incluir solo 2 decimales).
    # Ejemplo "Fichero respuesta" (primeras líneas)

    df=create_output_df(df)
    
    df.to_csv(path_output,sep='|',index=False,header=False)



In [ ]:
df_output=create_outputfile(output_data)

check prediction

# Nueva sección

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
target='Label'

In [ ]:
path_predictions='/content/predictions_0_22_03_13_v2.csv'

a=pd.read_csv(path_predictions)


In [ ]:
plot_results(a,a,stop=15)

In [ ]:
a['total']=150

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(a['total'],a['Label'],squared=False)

In [ ]:
a['error']=a['Label']-a['total']

In [ ]:
a

In [ ]:
y_true = [3, -0.5, 2, 7]
y_pred = [2.5, 0.0, 2, 8]

In [ ]:
errors=[]
for i in range(0,len(y_true)):
    error=y_true[i]-y_pred[i]
    errors.append(error)
errors

In [ ]:
0.5*0.5+0.5*0.5+1